In [1]:
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
import requests as r
from datetime import datetime

In [4]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
response = r.get(url) # put r.get(url) here(in first stages) to know (cotinues in next line)
# if there is a network error and prevent the function from furthur running
soup = BeautifulSoup(response.content, 'lxml')

In [8]:
def table_search(caption):
    """
    finding related 'table' tag in html document for the any 'caption' (part of table caption).
    the function takes part of a table caption(title) as a string 
    and returns the beautiful soup tag object for that table('table tag').
    the search is case insensitive. 
    it searches the beautiful soup object(response object from requeststhen converted to soup object).
    """
    caption_list= soup.find_all('caption') #returns list of table captions
    for element in caption_list:  # check the text inside each element of the list (cotinues in next line)
        # whether it matches the function argument
        if caption.lower() in element.string.lower():  # strings become case insensitive 
            return element.parent     
        # it returns the parent tag('table' tag) for the matching 'caption' tag 

In [223]:
def extract(): 
    
    ''' This function extracts the required
    information from the website, cleans the data and saves it to a dataframe. The
    function returns the dataframe for further processing. '''
    country=[]
    GDP_USD_millions=[]
    
    element_list = table_search('gdp').find_all('tr') 
    # 'i used function table_search() to (cotinues in next line)
    # look for the right tag 'table'. there are 7 'table' tags in the page now.
    #used 'gdp' as a keyword for the table. it can be any part of table title.
    
    for element in element_list[3:]: 
        #first two elements of the list are not informative (cotinues in next line)
        #so we start from third element at index=3
        row = element.find_all('td') # i used find_all one time in each loop 
        country.append(row[0].get_text(strip=True))
        GDP_USD_millions.append(row[2].string.replace(',','')) #removing ',' from text
        
    df = pd.DataFrame({"Country/Territory":country,"GDP_USD_millions":GDP_USD_millions})
    # df contain raw data and rows contain '—'. 
    # in the next lines we delete rows that contain'—'
    rows_with_dash=df[ lambda x: x['GDP_USD_millions'] == '—' ] #selecting rows with '—'
    index_to_drop=rows_with_dash.index  #index of rows with '—'
    return df.drop(index_to_drop).reset_index(drop=True) #dropping rows with '—' and resetting index

In [224]:
def transform(df):
    casted_column = df["GDP_USD_millions"].astype(int)
    df.update(round(casted_column/1000,2)) # input column should be the same as df column name for update
    df.rename(columns={"GDP_USD_millions": "GDP_USD_billions"}, inplace=True)
    return df

In [225]:
def load_to_csv(df, csv_path):
    df.to_csv(csv_path)

In [226]:
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

In [227]:
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

In [1]:
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format) 
    with open("./ETL_GDP_project_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [234]:
''' Here, i define the required entities(variables) and call the relevant 
functions in the correct order to complete the project.'''

db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

log_progress('Preliminaries complete. Initiating ETL process')
df = extract()
log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df)
log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')
sql_connection = sqlite3.connect('World_Economies.db')
log_progress('SQL Connection initiated.')
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')
query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)
log_progress('Process Complete.')
sql_connection.close()

SELECT * from Countries_by_GDP WHERE GDP_USD_billions >= 100
   Country/Territory  GDP_USD_billions
0      United States          26854.60
1              China          19373.59
2              Japan           4409.74
3            Germany           4308.85
4              India           3736.88
..               ...               ...
64             Kenya            118.13
65            Angola            117.88
66              Oman            104.90
67         Guatemala            102.31
68          Bulgaria            100.64

[69 rows x 2 columns]
